In [ ]:
# Necessary Imports
import cv2
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# Path to Kaggle Input
path = "D:/monument-prediction/Indian-monuments/images"
# Walk through the directory and list number of files
for dirpath, dirnames, filenames in os.walk(path):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
# append the training and the testing paths to the original path
train_dir =  path + "/train/"
test_dir = path + "/test/"
train_dir, test_dir

('D:/monument-prediction/Indian-monuments/images/train/',
 'D:/monument-prediction/Indian-monuments/images/test/')

In [ ]:
# get all the class names
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

array([], dtype=float64)

In [ ]:
# function used to view an random image
def view_random_image(target_dir, target_class):
    target_folder = target_dir  + target_class

  # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

  # Read in the image and plot it using matplotlib


    img = mpimg.imread(target_folder + "/" + random_image[0])
    imggs=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #converting the image to grayscale
    (thresh, imgbw)=cv2.threshold(imggs, 127, 255, cv2.THRESH_BINARY) #applying threshold operation to convert to balck and white
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");

    print(f"Image shape: {img.shape}") # show the shape of the image

    return img


# def to_grayscale_from_rgb(img):
#     img = tf.image.rgb_to_grayscale(img)
#     return img


def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
 #View a random image from the training dataset
import random
img = view_random_image(target_dir=train_dir, target_class=random.choice(class_names)) # get a random class name

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Rescale the data and create data generator instances
train_datagen = ImageDataGenerator(rescale=1/255,)
test_datagen = ImageDataGenerator(rescale=1/255,)




# Load data in from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(300, 300),
                                               batch_size=16,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                              target_size=(300, 300),
                                              batch_size=16,
                                              class_mode='categorical')

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Create our model
model_1 = Sequential([
  Conv2D(10, 5, activation='relu', input_shape=(300, 300, 3)),
  Conv2D(10, 5, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 5, activation='relu'),
  Conv2D(10, 5, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 5, activation='relu'),
  Conv2D(10, 5, activation='relu'),
#   MaxPool2D(),
#   Conv2D(10, 5, activation='relu'),
#   Conv2D(10, 5, activation='relu'),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(23, activation='softmax')
])

# Compile the model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=25,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data,
                        validation_steps=len(test_data))

In [ ]:
model_1.evaluate(test_data)

In [ ]:
model_1.save_weights("save_trained_weights.h5")

In [ ]:
plot_loss_curves(history_1)

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=300):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.

  return img

# Adjust function to work with multi-class
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
# make a new prediction
pred_and_plot(model_1, "D:/monument-prediction/Indian-monuments/images/test/charminar/35.jpg", class_names)

In [ ]:
# save the model
model_1.save("monument-model")

# **Creating an interactive UI for the Model**

*Before executing the below code blocks, please execute the first block of the file (Necessary Imports)*

# **Gradio**
Gradio is an Python library that simplifies and accelerates the process of building interactive machine learning and interfaces. It provides an easy-to-use platform for creating web-based UIs for your machine learning models, allowing users to interact with and make predictions using those models without needing any programming experience.

In [ ]:
# Installing gradio
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 12.

In [ ]:
# Cloning the repository to get access to data
!git clone https://github.com/developer-student-club-thapar/DSC-ML-Task

Cloning into 'DSC-ML-Task'...
remote: Enumerating objects: 3751, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 3751 (delta 8), reused 1 (delta 0), pack-reused 3736
Receiving objects: 100% (3751/3751), 577.95 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Updating files: 100% (4781/4781), done.


In [ ]:
%cd DSC-ML-Task/

/content/DSC-ML-Task


In [ ]:
# Load your trained model
model_path = "/content/DSC-ML-Task/saved_trained_model"
model = tf.keras.models.load_model(model_path)

In [ ]:
# Getting the class names
train_dir = "/content/DSC-ML-Task/Indian-monuments/images/train"  # Update with the path to your training directory
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))

In [ ]:
# Define a function for making predictions
def predict_image(input_image):
    # Preprocess the input image
    img = input_image

    # Resize the input image to match the model's expected shape (300x300)
    img = tf.image.resize(img, [300, 300])

    # Normalize the image
    img = img / 255.0

    # Make a prediction
    pred = model.predict(np.expand_dims(img, axis=0))

    # Get the predicted class
    predicted_class_index = np.argmax(pred)

    # Map the class index to a string label (replace with your class labels)
    predicted_class_label = class_names[predicted_class_index]

    return predicted_class_label

In [ ]:
import gradio as gr

# Create Gradio input component
input_component = gr.inputs.Image(type="numpy")

# Create a Gradio output component to display the predicted class label
output_component = gr.outputs.Label(type="text", label="Predicted Class Label")

# Create a Gradio interface
gr.Interface(fn=predict_image, inputs=input_component, outputs=output_component).launch()